In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/input-1/input.txt


In [109]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 .ABCTabcdefghiklmnoprstuvwyz
30


In [110]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[14, 15, 15, 1, 24, 14, 11, 22, 11]
hii there


In [111]:
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) 

torch.Size([1357143]) torch.int64
tensor([ 6, 14, 11,  1, 23, 25, 19,  1, 23, 14, 15, 19, 11, 23,  1,  8, 22, 15,
        13, 14, 24,  1, 15, 19,  1, 24, 14, 11,  1, 23, 16, 28,  2,  1,  6, 14,
        11,  1, 23, 25, 19,  1, 23, 14, 15, 19, 11, 23,  1,  8, 22, 15, 13, 14,
        24,  1, 15, 19,  1, 24, 14, 11,  1, 23, 16, 28,  2,  1,  5, 14, 15, 17,
        10, 22, 11, 19,  1, 21, 17,  7, 28,  1, 14,  7, 21, 21, 15, 17, 28,  1,
        15, 19,  1, 24, 14, 11,  1, 21,  7, 22, 16,  2,  1,  6, 14, 11,  1, 10,
        20, 13,  1,  8,  7, 22, 16, 11, 10,  1, 17, 20, 25, 10, 17, 28,  1,  7,
        24,  1, 19, 15, 13, 14, 24,  2,  1,  6, 14, 11,  1, 22,  7, 15, 19,  1,
        12,  7, 17, 17, 23,  1, 23, 20, 12, 24, 17, 28,  1, 20, 19,  1, 24, 14,
        11,  1, 13, 22, 20, 25, 19, 10,  2,  1,  6, 14, 11,  1, 23, 24,  7, 22,
        23,  1, 24, 27, 15, 19, 16, 17, 11,  1, 15, 19,  1, 24, 14, 11,  1,  9,
        17, 11,  7, 22,  1, 19, 15, 13, 14, 24,  1, 23, 16, 28,  2,  1,  3,  1,
      

In [112]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [113]:
block_size = 8
train_data[:block_size+1]

tensor([ 6, 14, 11,  1, 23, 25, 19,  1, 23])

In [114]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([6]) the target: 14
when input is tensor([ 6, 14]) the target: 11
when input is tensor([ 6, 14, 11]) the target: 1
when input is tensor([ 6, 14, 11,  1]) the target: 23
when input is tensor([ 6, 14, 11,  1, 23]) the target: 25
when input is tensor([ 6, 14, 11,  1, 23, 25]) the target: 19
when input is tensor([ 6, 14, 11,  1, 23, 25, 19]) the target: 1
when input is tensor([ 6, 14, 11,  1, 23, 25, 19,  1]) the target: 23


In [115]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[14,  7, 21, 21, 15, 17, 28,  1],
        [ 1,  6, 14, 11,  1, 23, 25, 19],
        [23,  2,  1,  6, 14, 11,  1, 24],
        [ 0,  0,  6, 14, 11,  1, 23, 25]])
targets:
torch.Size([4, 8])
tensor([[ 7, 21, 21, 15, 17, 28,  1, 15],
        [ 6, 14, 11,  1, 23, 25, 19,  1],
        [ 2,  1,  6, 14, 11,  1, 24, 22],
        [ 0,  6, 14, 11,  1, 23, 25, 19]])
----
when input is [14] the target: 7
when input is [14, 7] the target: 21
when input is [14, 7, 21] the target: 21
when input is [14, 7, 21, 21] the target: 15
when input is [14, 7, 21, 21, 15] the target: 17
when input is [14, 7, 21, 21, 15, 17] the target: 28
when input is [14, 7, 21, 21, 15, 17, 28] the target: 1
when input is [14, 7, 21, 21, 15, 17, 28, 1] the target: 15
when input is [1] the target: 6
when input is [1, 6] the target: 14
when input is [1, 6, 14] the target: 11
when input is [1, 6, 14, 11] the target: 1
when input is [1, 6, 14, 11, 1] the target: 23
when input is [1, 6, 14, 11, 1

In [116]:
print(xb)

tensor([[14,  7, 21, 21, 15, 17, 28,  1],
        [ 1,  6, 14, 11,  1, 23, 25, 19],
        [23,  2,  1,  6, 14, 11,  1, 24],
        [ 0,  0,  6, 14, 11,  1, 23, 25]])


In [117]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 30])
tensor(3.4631, grad_fn=<NllLossBackward0>)

ndoozbbnoh ywapesppb
hlcnuccrCnlTvBtbonczpyivwobsw
kcn tB.CgmmaetaCAlgTBsplAzmczh tCdlAnwtadgTBwuphv


In [118]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [119]:
batch_size = 32
for steps in range(5000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

1.7273764610290527


In [120]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))



eve sary farmain als A Thestarrs sththealrs  cared g. tapath



The d mapigheshe mA moung. ththe t thezerCChe doun g at. h panghire tht oghezeze Thesir t thevestlat. t. ildoge the heze onghtmoghe sontlontheen t. The The on n pppinghe mogrees. thtrksuds s s tA makle t tathtly ve wat. atrarCghe catatheally. tly cay sunain cap. ron tathat. t ont. A the The s g t s Bfzee ld
The mnght wfan hthematrt tappinhis matay.  s nky.
A e n g s ppzet len re y gThe sales ppTht t. llks t sigks wig. le the heacl mat htunt  the he Ththin hards d. cay pinge t. nilary tCBndofaCgrkaneve orker she  leThin het hed cal the cle sarnt. twightaveerkt. nd t dron taroftrdoA stbaininin seg in may th thin hees wandsat pilentmoft brk. arerks he A pin marn singhtun t t. st he ly. tappgrBighingrCpves tles ban Theshe in pin the s. in rdshentwalA caintat. ly Cfohet.hee is ly s t g. pllig t sthe aly. lesthare e dlmatrog t cle drigepin Bin Cze tlesat g. brang sun Thes ly A nczersthe Bintalesun bply thishininds ccay theds o

In [121]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [122]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [123]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [124]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [126]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [127]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [128]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [129]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [174]:
k.var()

tensor(1.0449)

In [175]:
q.var()

tensor(1.0700)

In [176]:
wei.var()

tensor(1.0918)

In [177]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [179]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [180]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [181]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [182]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [183]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=2000)[0].tolist()))


A mat maly trong. A s sog s fthe ararkeaCg gheve sthe twatllly. malkA dngheat talounghtwindsTh s. sondwary Thtly. carky. rondsthedoveveat she tbro

in hemoudly aveng Bing. ge The s Thtvenghevecn Thtlles atared. s ds manonk. bhe ing ig Thee are tandspanin ealogpat. heee The Thtbrken lky brntly.
Thlly ghthly t thbroudofa

brs ed. ily dog eninbrig t inky far on rezeanin montwin The 
Theestry. mat sun at. in 
rn htre Thtrindrnen ll mareat ThTheres stht brke she the tloud.oy n ay on in lernindsaleve br starain stherdrk. leve e s A stwinghmarCon hed. ightry ppig talle n ftat s leale Checapang. s at. stbappls loplonin ht. Cht on ing. appathalel d are t trkl A
The sthe tws moun s hetre in broun s  The matheeeve s Thinun ovTheve t g.alle ghe leatly thinghtht. s le A The l The ave taalind. s thong fingh Bun sCstre ly t alsthtunir
Ble The thanghtle g t s sun thee hundlly theds Athe d. apig.
arChe pl sound s san Then pin sar ig. A t s tverofkg Theshes. onds dlethtre Thy ig manBudsheat an s 
CBin 

In [185]:
import torch


context = "com"


context_ids = torch.tensor([encode(context)], dtype=torch.long)
print(f"Encoded context: {context_ids}")


output = m.generate(idx=context_ids, max_new_tokens=1)

next_char_id = output[0][-1].item()
next_char = decode([next_char_id])

print(f"Given '{context}' => Next character predicted: '{next_char}'")

Encoded context: tensor([[ 9, 20, 18]])
Given 'com' => Next character predicted: 'o'
